In [1]:
import os
import numpy as np
import random

In [2]:
path_dir = 'C://rnn_data' 
file_list = os.listdir(path_dir) 
# 20181016_135027_binary.txt 삭제 (no data)

In [3]:
file_list

['20181009_100023_binary.txt',
 '20181009_101055_binary.txt',
 '20181009_101420_binary.txt',
 '20181009_101635_binary.txt',
 '20181009_102248_binary.txt',
 '20181009_102508_binary.txt',
 '20181009_102849_binary.txt',
 '20181009_103752_binary.txt',
 '20181009_104216_binary.txt',
 '20181009_104446_binary.txt',
 '20181009_105554_binary.txt',
 '20181009_110458_binary.txt',
 '20181009_110901_binary.txt',
 '20181009_111450_binary.txt',
 '20181009_111734_binary.txt',
 '20181016_134334_binary.txt',
 '20181016_134532_binary.txt',
 '20181016_135124_binary.txt',
 '20181016_142025_binary.txt',
 '20181016_142531_binary.txt',
 '20181114_135427_binary.txt',
 '20181114_140400_binary.txt',
 '20181114_141258_binary.txt',
 '20181114_142450_binary.txt',
 '20181114_175502_binary.txt',
 '20181114_180324_binary.txt',
 '20181114_181316_binary.txt',
 'rnn_label.txt']

In [4]:
len(file_list)

28

In [5]:
def parse(raw_data):
    data = bytearray(raw_data)
       # print(len(data))
       # parse the sync and data signal in bytearray
    if len(data) < 2:
        return None, None
    if (data[0] >> 6) > 0: # '11'로 시작하는 데이터 일 때, 맨 앞의 데이터 삭제
        del data[:1]
    if len(data) % 2 == 1: # 전체 데이터 개수가 홀수인 경우
        del data[-1:]

    values = []
    sync = []
    for index in range(0, len(data), 2):
        if (data[index] >> 6) > 0 and index+2 <len(data): #내가 추가한 코드
            index_2 = index + 1
        else:
            index_2 = index
        high = data[index_2] & 0x1F
        low = data[index_2 + 1] & 0x1F
        values.append(high << 5 | low) 
        sync.append(True if (data[index_2] >> 5) == 1 else False)

    sync = sync #sync = np.array(sync)
    data = values #data = np.array(values)
        
    # print(self.sync, self.data, len(self.sync), len(self.data))
    return sync, data

In [6]:
sync_list = [] # 모든 파일들의 싱크 정보 리스트
data_list = [] # 모든 파일들의 데이터 정보 리스트
print('raw data - > data list, sync list')
for i in range(len(file_list) - 1):
    src_path = path_dir + '/' + file_list[i]
    data_file = open(src_path, "rb")
    read_data = data_file.read()
    sync, data = parse(read_data)
    sync_list.append(sync)
    data_list.append(data)
    print(i + 1, '번째 파일 완료')

raw data - > data list, sync list
1 번째 파일 완료
2 번째 파일 완료
3 번째 파일 완료
4 번째 파일 완료
5 번째 파일 완료
6 번째 파일 완료
7 번째 파일 완료
8 번째 파일 완료
9 번째 파일 완료
10 번째 파일 완료
11 번째 파일 완료
12 번째 파일 완료
13 번째 파일 완료
14 번째 파일 완료
15 번째 파일 완료
16 번째 파일 완료
17 번째 파일 완료
18 번째 파일 완료
19 번째 파일 완료
20 번째 파일 완료
21 번째 파일 완료
22 번째 파일 완료
23 번째 파일 완료
24 번째 파일 완료
25 번째 파일 완료
26 번째 파일 완료
27 번째 파일 완료


In [7]:
for index in range(len(file_list) - 1):
    print(index+1, 'True count:', sync_list[index].count(True))
    print(index+1, 'False count:', sync_list[index].count(False))
    print("")

1 True count: 896904
1 False count: 891006

2 True count: 346619
2 False count: 415441

3 True count: 238071
3 False count: 236751

4 True count: 279501
4 False count: 277389

5 True count: 209256
5 False count: 330047

6 True count: 229316
6 False count: 227920

7 True count: 235488
7 False count: 233472

8 True count: 343957
8 False count: 341897

9 True count: 285129
9 False count: 283485

10 True count: 305603
10 False count: 304045

11 True count: 291238
11 False count: 289100

12 True count: 438147
12 False count: 435291

13 True count: 244121
13 False count: 312769

14 True count: 238248
14 False count: 236574

15 True count: 405666
15 False count: 403289

16 True count: 120993
16 False count: 119349

17 True count: 537251
17 False count: 530759

18 True count: 324563
18 False count: 320257

19 True count: 337077
19 False count: 372224

20 True count: 268558
20 False count: 264884

21 True count: 985626
21 False count: 972282

22 True count: 1271793
22 False count: 1254729

23 T

In [8]:
len(data_list[4])

539303

In [9]:
data_list_seconds = [] # 모든 파일들의 데이터 정보(초단위)
for i in range(len(file_list) - 1):
    sync = sync_list[i]
    data = data_list[i]

    true_data  = []
    temp_d = []
    count = 0
    
    for index, tf_value in enumerate(sync):
        if tf_value == True:
            count += 1
            temp_d.append(data[index])
            if count == 2931:
                true_data.append(temp_d)
                count = 0
                temp_d = []
    
    data_list_seconds.append(true_data)
    print(i + 1, '번째 파일 완료, 길이:' ,len(data_list_seconds[i]),'초', end='')
    print(' ('+file_list[i]+')')
    
#앞의 리스트는 프리해줘도 좋음

1 번째 파일 완료, 길이: 306 초 (20181009_100023_binary.txt)
2 번째 파일 완료, 길이: 118 초 (20181009_101055_binary.txt)
3 번째 파일 완료, 길이: 81 초 (20181009_101420_binary.txt)
4 번째 파일 완료, 길이: 95 초 (20181009_101635_binary.txt)
5 번째 파일 완료, 길이: 71 초 (20181009_102248_binary.txt)
6 번째 파일 완료, 길이: 78 초 (20181009_102508_binary.txt)
7 번째 파일 완료, 길이: 80 초 (20181009_102849_binary.txt)
8 번째 파일 완료, 길이: 117 초 (20181009_103752_binary.txt)
9 번째 파일 완료, 길이: 97 초 (20181009_104216_binary.txt)
10 번째 파일 완료, 길이: 104 초 (20181009_104446_binary.txt)
11 번째 파일 완료, 길이: 99 초 (20181009_105554_binary.txt)
12 번째 파일 완료, 길이: 149 초 (20181009_110458_binary.txt)
13 번째 파일 완료, 길이: 83 초 (20181009_110901_binary.txt)
14 번째 파일 완료, 길이: 81 초 (20181009_111450_binary.txt)
15 번째 파일 완료, 길이: 138 초 (20181009_111734_binary.txt)
16 번째 파일 완료, 길이: 41 초 (20181016_134334_binary.txt)
17 번째 파일 완료, 길이: 183 초 (20181016_134532_binary.txt)
18 번째 파일 완료, 길이: 110 초 (20181016_135124_binary.txt)
19 번째 파일 완료, 길이: 115 초 (20181016_142025_binary.txt)
20 번째 파일 완료, 길이: 91 초 (20181016

In [10]:
len(data_list_seconds[4])

71

In [11]:
label_path_dir = path_dir + '/' + 'rnn_label.txt'
label_file = open(label_path_dir, 'r')

In [12]:
label_lines = label_file.readlines()

In [13]:
len(label_lines)

27

In [14]:
label_lines

['20181009_100023_binary.txt: w/ a person (15~28, 28~36, 151~162, 164~174, 237~253, 257~270)\n',
 '20181009_101055_binary.txt: w/ a person (38~70, 84~110)\n',
 '20181009_101420_binary.txt: w/ a person (18~36, 53~69)\n',
 '20181009_101635_binary.txt: w/ a person, w/ an iron plate (20~52, 62~83)\n',
 '20181009_102248_binary.txt: w/ a car (42~58)\n',
 '20181009_102508_binary.txt: w/ a car (28~36, 57~59)\n',
 '20181009_102849_binary.txt: w/ a car (25~32, 51~57)\n',
 '20181009_103752_binary.txt: w/ a person, w/ an iron plate (19~69, 69~107)\n',
 '20181009_104216_binary.txt: w/ a person, w/ an iron plate (8~47, 58~92)\n',
 '20181009_104446_binary.txt: w/ a person, w/ an iron plate (first), w/o the iron plate (second) (10~31, 41~57, 59~76, 85~98)\n',
 '20181009_105554_binary.txt: w/ a person (21~26, 26~30, 38~65, 65~92)\n',
 '20181009_110458_binary.txt: w/ a drone (DJI Phantom), w/o reflection tape (1~9, 36~47, 62~87, 106~115)\n',
 '20181009_110901_binary.txt: w/ a drone (DJI Phantom), w/o re

In [15]:
for line in label_lines: #파일 제일 마지막에 엔터 키 필요
    print(line, end='')
    print(line.split(" a ")[1].split(" ")[0].replace(",", "")) # person, car ,drone 분류 코드
    if line.count('(') == 0:
        print('전구간')
    else:
        print(line.split("(")[line.count('(')].split(")")[0].split(", ")) # 시간 부분만 추출
    print("")

20181009_100023_binary.txt: w/ a person (15~28, 28~36, 151~162, 164~174, 237~253, 257~270)
person
['15~28', '28~36', '151~162', '164~174', '237~253', '257~270']

20181009_101055_binary.txt: w/ a person (38~70, 84~110)
person
['38~70', '84~110']

20181009_101420_binary.txt: w/ a person (18~36, 53~69)
person
['18~36', '53~69']

20181009_101635_binary.txt: w/ a person, w/ an iron plate (20~52, 62~83)
person
['20~52', '62~83']

20181009_102248_binary.txt: w/ a car (42~58)
car
['42~58']

20181009_102508_binary.txt: w/ a car (28~36, 57~59)
car
['28~36', '57~59']

20181009_102849_binary.txt: w/ a car (25~32, 51~57)
car
['25~32', '51~57']

20181009_103752_binary.txt: w/ a person, w/ an iron plate (19~69, 69~107)
person
['19~69', '69~107']

20181009_104216_binary.txt: w/ a person, w/ an iron plate (8~47, 58~92)
person
['8~47', '58~92']

20181009_104446_binary.txt: w/ a person, w/ an iron plate (first), w/o the iron plate (second) (10~31, 41~57, 59~76, 85~98)
person
['10~31', '41~57', '59~76', '

In [16]:
labels = {'others': [], 'person': [], 'drone': [], 'car': []}

In [17]:
for index,line in enumerate(label_lines): #파일 제일 마지막에 엔터 키 필요
    label_name = line.split(" a ")[1].split(" ")[0].replace(",", "") # person, car ,drone 분류 코드
    if line.count('(') == 0:
        time = 'all'
    else:
        time = line.split("(")[line.count('(')].split(")")[0].split(", ") # 시간 부분만 추출
    
    if time == 'all':
        for now in range(0, len(data_list_seconds[index])):
            labels[label_name].append(data_list_seconds[index][now])
    else:
        for i, t in enumerate(time):
            start = int(t.split('~')[0]) # 초단위기 때문에 배열에 넣을때는 1을 빼서 넣어준다
            end = int(t.split('~')[1])
            if end > len(data_list_seconds[index])-1 :
                end = len(data_list_seconds[index])-1
            if i==0: #각파일에서 어떤 물체가 나타나기 전의 구간을 ohters로 정의
                for now in range(0, start-1):
                    labels['others'].append(data_list_seconds[index][now])
            for now in range(start-1, end):
                labels[label_name].append(data_list_seconds[index][now])
            

In [18]:
for i in range(15, 45):
    labels['others'].append(data_list_seconds[25][i]) #26번째 파일에서 보충
for i in range(60, 82):
    labels['others'].append(data_list_seconds[25][i])
for i in range(97, 123):
    labels['others'].append(data_list_seconds[25][i])

In [19]:
len(labels['others'])

500

In [20]:
len(labels['person'])

1071

In [21]:
len(labels['drone'])

552

In [22]:
len(labels['car'])

460

In [23]:
len(labels['drone'][3])

2931

In [36]:
labels['others'][15]

[512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 511,
 511,
 511,
 511,
 511,
 511,
 511,
 511,
 511,
 511,
 510,
 510,
 510,
 510,
 510,
 510,
 510,
 510,
 509,
 509,
 509,
 509,
 509,
 508,
 509,
 508,
 509,
 509,
 508,
 508,
 508,
 508,
 508,
 508,
 508,
 508,
 507,
 508,
 508,
 508,
 508,
 508,
 507,
 513,
 514,
 513,
 513,
 513,
 513,
 513,
 513,
 513,
 513,
 513,
 513,
 513,
 513,
 513,
 513,
 513,
 513,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 511,
 511,
 511,
 511,
 511,
 511,
 511,
 511,
 511,
 510,
 510,
 510,
 510,
 510,
 510,
 510,
 510,
 510,
 510,
 509,
 509,
 509,
 508,
 509,
 508,
 509,
 508,
 508,
 508,
 508,
 508,
 508,
 508,
 508,
 508,
 507,
 508,
 508,
 508,
 508,
 508,
 514,
 515,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514,
 514

In [24]:
others_arr = np.array(labels['others'])
others_arr.shape

(500, 2931)

In [25]:
person_arr = np.array(labels['person'])
person_arr.shape

(1071, 2931)

In [26]:
drone_arr = np.array(labels['drone'])
drone_arr.shape

(552, 2931)

In [27]:
car_arr = np.array(labels['car'])
car_arr.shape

(460, 2931)

In [28]:
type(drone_arr)

numpy.ndarray

In [29]:
train_input = []
train_label = []

#0 - others
#1 - person
#2 - car
#3 - uav

for i in range(int(len(labels['car'])*0.7)): #가장 길이가 작은 데이터에 맞게 자름
    train_input.append(labels['others'][i][:2916])
    train_label.append(0)
    
    train_input.append(labels['person'][i][:2916])
    train_label.append(1)
    
    train_input.append(labels['car'][i][:2916])
    train_label.append(2)
    
    train_input.append(labels['drone'][i][:2916])
    train_label.append(3)

# 54*54 = 2916

tmp = [[x,y] for x,y in zip(train_input, train_label)]
random.shuffle(tmp)
train_input = [n[0] for n in tmp]
train_label = [n[1] for n in tmp]

train_input = np.reshape(train_input, (-1, 54*54))
#train_label = np.reshape(train_label, (-1, 4))
train_input = np.array(train_input).astype(np.float32)
#train_label = np.array(train_label).astype(np.float32)
train_label = np.array(train_label).astype(np.int32)

np.save("rnn_train_input.npy", train_input)
np.save("rnn_train_label.npy", train_label)
print('rnn train data 생성')
    
test_input = []
test_label = []
    
for i in range(int(len(labels['car'])*0.7), len(labels['car'])):
    test_input.append(labels['others'][i][:2916])
    test_label.append(0)
    
    test_input.append(labels['person'][i][:2916])
    test_label.append(1)
    
    test_input.append(labels['car'][i][:2916])
    test_label.append(2)
    
    test_input.append(labels['drone'][i][:2916])
    test_label.append(3)

test_input = np.reshape(test_input, (-1, 54*54))
#test_label = np.reshape(test_label, (-1, 4))
test_input = np.array(test_input).astype(np.float32)
#test_label = np.array(test_label).astype(np.float32)
test_label = np.array(test_label).astype(np.int32)

np.save("rnn_test_input.npy", test_input)
np.save("rnn_test_label.npy", test_label)               
               

print('test data 생성')

rnn train data 생성
test data 생성


In [30]:
test_label.shape

(552,)

In [31]:
test_label[2]

2

In [32]:
labels['person'][25]

[470,
 469,
 468,
 468,
 467,
 467,
 466,
 465,
 465,
 464,
 464,
 463,
 463,
 462,
 461,
 461,
 462,
 461,
 461,
 461,
 460,
 460,
 460,
 460,
 460,
 460,
 460,
 460,
 460,
 459,
 459,
 459,
 458,
 458,
 457,
 457,
 457,
 456,
 456,
 455,
 455,
 454,
 453,
 452,
 451,
 450,
 449,
 448,
 447,
 446,
 445,
 444,
 443,
 442,
 442,
 440,
 440,
 439,
 438,
 438,
 437,
 436,
 436,
 435,
 434,
 434,
 434,
 434,
 433,
 433,
 432,
 432,
 431,
 431,
 430,
 430,
 430,
 430,
 429,
 429,
 429,
 428,
 428,
 427,
 427,
 426,
 426,
 425,
 424,
 424,
 424,
 423,
 423,
 422,
 421,
 420,
 420,
 419,
 419,
 419,
 418,
 418,
 417,
 417,
 415,
 415,
 414,
 413,
 412,
 412,
 411,
 410,
 409,
 408,
 407,
 405,
 405,
 404,
 403,
 401,
 400,
 398,
 396,
 394,
 392,
 391,
 389,
 388,
 385,
 383,
 381,
 379,
 377,
 376,
 374,
 372,
 471,
 470,
 469,
 469,
 468,
 467,
 467,
 466,
 465,
 465,
 465,
 464,
 464,
 463,
 463,
 462,
 462,
 462,
 462,
 461,
 461,
 461,
 461,
 461,
 460,
 461,
 460,
 460,
 460,
 460,
 460